In [ ]:
import pytesseract
from PIL import ImageGrab, Image
import pyautogui
import cv2
import numpy as np
import time

# Define the regions of the screen for multipliers and blue bar
multiplier_region = (x1, y1, width1, height1)  # Replace with coordinates of the multiplier region
blue_bar_region = (x2, y2, width2, height2)    # Replace with coordinates of the blue bar region

# Initialize counters and flags
space_pressed = False
s_press_count = 0

def capture_screen(region):
    # Capture the specified screen region
    screenshot = ImageGrab.grab(bbox=region)
    return screenshot

def has_blue_bar(image):
    # Check for the blue bar using color detection
    img_cv = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    hsv_img = cv2.cvtColor(img_cv, cv2.COLOR_BGR2HSV)
    lower_blue = np.array([100, 150, 150])
    upper_blue = np.array([130, 255, 255])
    blue_mask = cv2.inRange(hsv_img, lower_blue, upper_blue)
    blue_pixels = cv2.countNonZero(blue_mask)
    return blue_pixels > threshold_value  # Adjust threshold_value based on testing

def has_green_background(image):
    # Check for green background on multipliers
    img_cv = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    hsv_img = cv2.cvtColor(img_cv, cv2.COLOR_BGR2HSV)
    lower_green = np.array([40, 40, 40])
    upper_green = np.array([70, 255, 255])
    green_mask = cv2.inRange(hsv_img, lower_green, upper_green)
    green_pixels = cv2.countNonZero(green_mask)
    return green_pixels > threshold_value  # Adjust based on testing

def extract_multipliers(image):
    # Extract multipliers using OCR
    text = pytesseract.image_to_string(image, config='--psm 6')
    try:
        multipliers = [float(num) for num in text.split('x') if num.strip()]
        return multipliers
    except ValueError:
        return []

def main():
    global space_pressed, s_press_count
    
    while True:
        # Capture blue bar and multiplier regions
        blue_bar_screenshot = capture_screen(blue_bar_region)
        multiplier_screenshot = capture_screen(multiplier_region)
        
        # Check for the existence of the blue bar
        blue_bar_exists = has_blue_bar(blue_bar_screenshot)
        
        # Extract multipliers and check for green background
        multipliers = extract_multipliers(multiplier_screenshot)
        green_background_present = any(has_green_background(multiplier_screenshot.crop(box)) for box in multiplier_boxes)

        # Condition to press space: no green multipliers, blue bar exists, and space hasn't been pressed yet
        if not green_background_present and blue_bar_exists and not space_pressed:
            pyautogui.press('space')
            space_pressed = True  # Mark space as pressed
            s_press_count += 1  # Count each time "s" will be pressed
            time.sleep(0.1)  # Short delay for reliability
            pyautogui.press('s')  # Follow up by pressing "s"
            print("Pressed space and 's'")
        
        # Condition to press "a" based on green background presence
        elif green_background_present and s_press_count > 0:
            for _ in range(s_press_count):
                pyautogui.press('a')
                time.sleep(0.1)  # Small delay for each "a" press
            print(f"Pressed 'a' {s_press_count} times")
            s_press_count = 0  # Reset "s" count after pressing "a"

        # Reset condition when page refreshes and displays new multipliers
        if not green_background_present and not blue_bar_exists and space_pressed:
            space_pressed = False  # Allow pressing space again
            print("Reset space press due to page refresh")

        # Add a short delay to avoid excessive CPU usage
        time.sleep(0.5)

# Run the bot
main()


In [24]:
import pyautogui
import time

print("Starting to track mouse position. Press Ctrl+C to stop.")
try:
    while True:
        x, y = pyautogui.position()
        print(f"Mouse position: ({x}, {y})", end='\r')
        time.sleep(0.1)
except KeyboardInterrupt:
    print("\nStopped tracking.")


Starting to track mouse position. Press Ctrl+C to stop.
Mouse position: (4581, -846)
Stopped tracking.


In [26]:
import pyautogui
from PIL import Image
import numpy as np
import cv2
import pytesseract
import time
from collections import deque
import threading

# If Tesseract is not in your PATH, uncomment and set the path
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# ------------------- Configuration -------------------

# Define the coordinates for multipliers and the blue bar
# Format: (left, top, width, height)
MULTIPLIER_AREAS = [
    # (1640, 375, 80, 45),  # Multiplier 1 (commented out)
    (1730, 375, 75, 20),  # Multiplier 2
    (1825, 375, 75, 20),  # Multiplier 3
    (1910, 375, 75, 20),  # Multiplier 4
    (1995, 375, 75, 20),  # Multiplier 5
    # Add more as needed
]

MULTIPLIER_TEXT_AREAS = [
    # (1640, 375, 80, 45),  # Multiplier 1 Text (commented out)
    (1730, 375, 80, 45),  # Multiplier 2 Text
    (1815, 375, 80, 45),  # Multiplier 3 Text
    (1905, 375, 80, 45),  # Multiplier 4 Text
    (1990, 375, 80, 45),  # Multiplier 5 Text
    # Add more as needed
]

BLUE_BAR_AREA = (895, 1220, 625, 10)  # Coordinates for the blue bar

# Define color ranges in HSV
# These ranges might need to be adjusted based on actual colors
GREEN_LOWER = np.array([50, 100, 100])
GREEN_UPPER = np.array([70, 255, 255])

BLUE_LOWER = np.array([100, 150, 0])
BLUE_UPPER = np.array([140, 255, 255])

# Initialize deque to keep track of last 4 multipliers
last_four_multipliers = deque(maxlen=4)

# Initialize deque to track the history of multiplier sets to prevent duplicate actions
previous_multiplier_sets = deque(maxlen=1)

# Counter for 's' presses
s_press_count = 0

# Lock for thread-safe operations
lock = threading.Lock()

# ------------------- Helper Functions -------------------

def is_color_present(area, lower_hsv, upper_hsv):
    """
    Check if the specified color is present in the given screen area.
    """
    screenshot = pyautogui.screenshot(region=area)
    img = cv2.cvtColor(np.array(screenshot), cv2.COLOR_RGB2BGR)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower_hsv, upper_hsv)
    return cv2.countNonZero(mask) > 0

def read_multiplier(area):
    """
    Read the multiplier value from the specified screen area using OCR.
    """
    screenshot = pyautogui.screenshot(region=area)
    gray = cv2.cvtColor(np.array(screenshot), cv2.COLOR_RGB2GRAY)
    # Revert to previous OCR parameters for better accuracy
    # Apply threshold to make text more readable
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV)
    # Use pytesseract to extract text
    config = '--psm 7 -c tessedit_char_whitelist=0123456789.xX'
    text = pytesseract.image_to_string(thresh, config=config)
    # Clean the text
    text = text.strip().replace('x', '').replace('X', '').replace(' ', '')
    try:
        multiplier = float(text)
        return multiplier
    except ValueError:
        return None

def get_multipliers():
    """
    Retrieve the current multipliers by checking background colors and reading text.
    """
    multipliers = []
    for idx, (color_area, text_area) in enumerate(zip(MULTIPLIER_AREAS, MULTIPLIER_TEXT_AREAS), start=2):
        green = is_color_present(color_area, GREEN_LOWER, GREEN_UPPER)
        if green:
            # If green background is detected, infer multiplier >=2x
            inferred_multiplier = 2.0  # Adjust if higher precision is needed
            multipliers.append(inferred_multiplier)
            print(f"Multiplier {idx}: 2x or more")
        else:
            # Attempt to read the multiplier using OCR
            multiplier = read_multiplier(text_area)
            if multiplier is not None:
                multipliers.append(multiplier)
                print(f"Multiplier {idx}: {multiplier}x.")
            else:
                # Failed to read multiplier and no green background; skip adding
                print(f"Multiplier {idx}: Failed to read and no green background.")
    return multipliers

def are_multipliers_same(current, previous):
    """
    Check if the current set of multipliers is the same as the previous set.
    """
    return list(current) == list(previous)

# ------------------- Main Bot Logic -------------------

def main():
    global s_press_count
    print("Starting the bot. Press Ctrl+C to stop.")
    try:
        while True:
            # Retrieve current multipliers
            multipliers = get_multipliers()

            # Check if enough multipliers are read
            if len(multipliers) < 4:
                print("Not enough multipliers read. Waiting for next iteration.")
                print("-" * 50)
                time.sleep(5)
                continue

            # Check if current multipliers are the same as the previous set
            with lock:
                if previous_multiplier_sets and are_multipliers_same(multipliers, previous_multiplier_sets[-1]):
                    print("Multipliers are the same as the previous set. No action taken.")
                    print("-" * 50)
                    time.sleep(10)
                    continue
                else:
                    # Update the previous_multiplier_sets deque
                    previous_multiplier_sets.append(multipliers.copy())

            # Update last four multipliers
            with lock:
                for m in multipliers:
                    last_four_multipliers.append(m)

            # Debugging output
            print(f"Last four multipliers: {list(last_four_multipliers)}")

            # Check for blue bar presence
            blue_bar_present = is_color_present(BLUE_BAR_AREA, BLUE_LOWER, BLUE_UPPER)
            print(f"Blue bar present: {blue_bar_present}")

            # Implement the clicking logic
            if blue_bar_present:
                # Check if the last four multipliers are all below 2
                if len(last_four_multipliers) == 4 and all(m < 2 for m in last_four_multipliers):
                    # Press 'space' and 's' once
                    pyautogui.moveTo(245,30)
                    pyautogui.click()
                    time.sleep(0.2)
                    pyautogui.press('space')
                    time.sleep(1)
                    pyautogui.press('s')
                    with lock:
                        s_press_count += 1
                    print("Pressed 'space' and 's'")
                else:
                    # If a multiplier >=2 is detected and 's' was pressed before
                    if s_press_count > 0:
                        for _ in range(s_press_count):
                            pyautogui.moveTo(245,30)
                            pyautogui.click()
                            time.sleep(0.2)
                            pyautogui.press('a')
                            print("Pressed 'a'")
                            time.sleep(0.1)  # Small delay between presses
                        with lock:
                            s_press_count = 0  # Reset the counter
                time.sleep(10)
            else:
                print("Blue bar not present. Waiting for countdown.")
                time.sleep(10)  # Adjust the sleep time as needed based on game speed

            print("-" * 50)
            # time.sleep(10)  # Adjust the sleep time as needed based on game speed

    except KeyboardInterrupt:
        print("Bot stopped by user.")

if __name__ == "__main__":
    main()


Starting the bot. Press Ctrl+C to stop.
Multiplier 2: 2x or more
Multiplier 3: 2x or more
Multiplier 4: 2x or more
Multiplier 5: 2x or more
Last four multipliers: [2.0, 2.0, 2.0, 2.0]
Blue bar present: False
Blue bar not present. Waiting for countdown.
--------------------------------------------------
Multiplier 2: 2x or more
Multiplier 3: 2x or more
Multiplier 4: 2x or more
Multiplier 5: 1.62x.
Last four multipliers: [2.0, 2.0, 2.0, 1.62]
Blue bar present: True
--------------------------------------------------
Multiplier 2: 2x or more
Multiplier 3: 2x or more
Multiplier 4: 2x or more
Multiplier 5: 1.62x.
Multipliers are the same as the previous set. No action taken.
--------------------------------------------------
Multiplier 2: 2x or more
Multiplier 3: 2x or more
Multiplier 4: 2x or more
Multiplier 5: 1.62x.
Multipliers are the same as the previous set. No action taken.
--------------------------------------------------
Multiplier 2: 2x or more
Multiplier 3: 2x or more
Multiplier 